In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch as th
from torch import nn
from torch.nn import functional as F
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
MAIN_PATH = '/content/gdrive/MyDrive/neuroAI/emotion prediction'
DATA_PATH = os.path.join(MAIN_PATH, 'data')
os.chdir(MAIN_PATH)

In [ ]:
emotions = pd.read_csv(os.path.join(DATA_PATH, 'emotions.csv')).dropna()

In [ ]:
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
emotions.head(10)

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.60,26.3,1.070,0.411,-15.70,2.06,3.150,...,23.50,20.3,20.3,23.50,-215.00,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.80,22.8,6.550,1.680,2.88,3.83,-4.820,...,-23.30,-21.8,-21.8,-23.30,182.00,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.70,23.7,79.900,3.360,90.20,89.90,2.030,...,462.00,-233.0,-233.0,462.00,-267.00,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.80,24.3,-0.584,-0.284,8.82,2.30,-1.970,...,299.00,-243.0,-243.0,299.00,132.00,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.30,24.5,34.800,-5.790,3.06,41.40,5.520,...,12.00,38.1,38.1,12.00,119.00,-17.60,23.90,23.90,-17.60,NEUTRAL
5,31.00,30.9,29.6,28.50,24.0,1.650,1.540,3.83,1.87,-1.210,...,-1.48,30.2,30.2,-1.48,134.00,3.59,-12.70,-12.70,3.59,NEUTRAL
6,10.80,21.0,44.7,4.87,28.1,2.140,1.020,13.20,1.16,-4.390,...,-15.60,-41.0,-41.0,-15.60,89.50,40.60,-55.20,-55.20,40.60,POSITIVE
7,17.80,27.8,-102.0,16.90,26.9,-3.210,-1.950,9.80,-3.24,-0.955,...,-177.00,32.8,32.8,-177.00,-417.00,384.00,-186.00,-186.00,384.00,NEGATIVE
8,11.50,29.7,34.9,10.20,26.9,-38.000,-1.650,3.89,-33.50,-3.300,...,-8.38,38.7,38.7,-8.38,115.00,-7.00,3.20,3.20,-7.00,NEUTRAL
9,8.91,29.2,-314.0,6.51,30.9,-1.880,1.900,11.90,-3.60,5.700,...,226.00,-81.8,-81.8,226.00,1.84,99.40,-40.30,-40.30,99.40,NEGATIVE


In [ ]:
emotions['label'].value_counts()

NEUTRAL     716
NEGATIVE    708
POSITIVE    708
Name: label, dtype: int64

In [ ]:
def data(emotions: pd.DataFrame) -> tuple:
    y = emotions['label'].replace(
        {
            'NEUTRAL': 1,
            'NEGATIVE': 0,
            'POSITIVE': 2
        }
    )
    X = emotions.drop('label', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
    return X_train.values, X_test.values, y_train.values, y_test.values

In [ ]:
X_train, X_test, y_train, y_test = data(emotions)

In [ ]:
X_train

array([[  29.  ,   32.  ,   32.1 , ...,   38.7 ,   38.7 ,   -3.18],
       [  13.2 ,   31.4 , -795.  , ..., -269.  , -269.  ,  698.  ],
       [   7.18,   30.6 ,   26.2 , ...,  -28.1 ,  -28.1 ,  -29.6 ],
       ...,
       [   5.  ,   24.  , -377.  , ..., -101.  , -101.  ,  254.  ],
       [  15.1 ,   32.4 , -146.  , ...,    5.34,    5.34,  -85.2 ],
       [ 304.  , -112.  ,  633.  , ..., -320.  , -320.  ,  824.  ]])

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
            return len(self.y)
    def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

In [ ]:
train_data = Data(X_train, y_train)
test_data = Data(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
class EmotionModel(nn.Module):
    def __init__(self, input_size):
        super(EmotionModel, self).__init__()
        self.input_size = input_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=64)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = x.view(len(x), 1, -1)
        x, _ = self.lstm(x)
        x = self.flatten(x)
        x = nn.Linear(in_features=x.size()[1], out_features=3)(x)
        return self.softmax(x)

In [ ]:
model = EmotionModel(X_train.shape[1])
loss_fn = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model.parameters(), lr=1e-4)

N_EPOCHS = 100

dataset_size = len(train_loader)

for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch + 1}\n-------------------------------")

    for id_batch, (x_batch, y_batch) in enumerate(train_loader):
        y_batch = y_batch.to(device=device, dtype=th.int64)
        
        y_batch_pred = model(x_batch.float())
        
        loss = loss_fn(y_batch_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

Epoch 1
-------------------------------
loss: 1.089301  [   32/   54]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Epoch 2
-------------------------------
loss: 1.112937  [   32/   54]
Epoch 3
-------------------------------
loss: 1.105464  [   32/   54]
Epoch 4
-------------------------------
loss: 1.060115  [   32/   54]
Epoch 5
-------------------------------
loss: 1.152818  [   32/   54]
Epoch 6
-------------------------------
loss: 1.109928  [   32/   54]
Epoch 7
-------------------------------
loss: 1.116979  [   32/   54]
Epoch 8
-------------------------------
loss: 1.086081  [   32/   54]
Epoch 9
-------------------------------
loss: 1.107259  [   32/   54]
Epoch 10
-------------------------------
loss: 1.086468  [   32/   54]
Epoch 11
-------------------------------
loss: 1.124497  [   32/   54]
Epoch 12
-------------------------------
loss: 1.114032  [   32/   54]
Epoch 13
-------------------------------
loss: 1.081601  [   32/   54]
Epoch 14
-------------------------------
loss: 1.092453  [   32/   54]
Epoch 15
-------------------------------
loss: 1.060108  [   32/   54]
Epoch 16
-----

In [ ]:
running_accuracy = [] 
total = 0 
loss_fn = nn.CrossEntropyLoss()

for id_batch, (x_batch, y_batch) in enumerate(test_loader):
    y_batch = y_batch.to(device=device, dtype=th.int64)
    
    y_batch_pred = model(x_batch.float())
    
    loss = loss_fn(y_batch_pred, y_batch)
    y_pred = th.argmax(y_batch_pred, dim=1)
    running_accuracy.append(th.sum(y_pred == y_batch) / y_batch.size(0))
print('accuracy: ', sum(running_accuracy)/len(running_accuracy))

accuracy:  tensor(0.3267)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
